# Final Group Project NLP - Part 5

* Ahmed Mohamed Elghamry Shehata
* Ahmed Mahmoud Abdelmoneim Abdelhamid
* Noureldin Mohamed Abdelsalm Mohamed Hamedo
* Sergio Rodrigo Fernandez Testa

# BERT Categorizer

In [ ]:
!gdown 1sST4xH7AbvoaDYmMyns7oJqsr7WppbQg

Downloading...
From (original): https://drive.google.com/uc?id=1sST4xH7AbvoaDYmMyns7oJqsr7WppbQg
From (redirected): https://drive.google.com/uc?id=1sST4xH7AbvoaDYmMyns7oJqsr7WppbQg&confirm=t&uuid=331a79d5-32cc-47e0-97fd-35817b8e83b4
To: /content/NLP_data.zip
100% 725M/725M [00:13<00:00, 52.2MB/s]


In [ ]:
!unzip NLP_data.zip

Archive:  NLP_data.zip
   creating: content/drive/MyDrive/NLP data/
   creating: content/drive/MyDrive/NLP data/recipe_classifier/
  inflating: content/drive/MyDrive/NLP data/recipe_classifier/tokenizer_config.json  
  inflating: content/drive/MyDrive/NLP data/recipe_classifier/special_tokens_map.json  
  inflating: content/drive/MyDrive/NLP data/recipe_classifier/vocab.txt  
  inflating: content/drive/MyDrive/NLP data/Clustered_dataset.csv  
  inflating: content/drive/MyDrive/NLP data/test_encodings.pkl  
  inflating: content/drive/MyDrive/NLP data/train_encodings.pkl  
  inflating: content/drive/MyDrive/NLP data/test_labels.csv  
  inflating: content/drive/MyDrive/NLP data/train_labels.csv  


In [ ]:
import pandas as pd
import re
import ast

In [ ]:
data_folder = "/content/content/drive/MyDrive/NLP data"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer
tokenizer = AutoTokenizer.from_pretrained(f"{data_folder}/recipe_classifier")

In [ ]:
import joblib
train_encodings = joblib.load(f"{data_folder}/train_encodings.pkl")
test_encodings = joblib.load(f"{data_folder}/test_encodings.pkl")

In [ ]:
import pandas as pd
train_labels = pd.read_csv(f"{data_folder}/train_labels.csv")
train_labels = train_labels['label'].tolist()

test_labels = pd.read_csv(f"{data_folder}/test_labels.csv")
test_labels = test_labels['label'].tolist()

In [ ]:
import torch

class RecipeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RecipeDataset(train_encodings, train_labels)
test_dataset = RecipeDataset(test_encodings, test_labels)


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Subset
import numpy as np

# Subset size
N = 30_000
N_eval = 10_000

# Create consistent random indices
train_indices = np.random.choice(len(train_dataset), N, replace=False)
eval_indices = np.random.choice(len(test_dataset), N_eval, replace=False)

# Subsets
train_subset = Subset(train_dataset, train_indices)
eval_subset = Subset(test_dataset, eval_indices)

# Load model
num_labels = 2
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
)

# Train!
trainer.train()


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.104900,0.065670
2,0.046500,0.058063
3,0.007000,0.075414


TrainOutput(global_step=5625, training_loss=0.05214171215856655, metrics={'train_runtime': 1276.9642, 'train_samples_per_second': 70.48, 'train_steps_per_second': 4.405, 'total_flos': 2980516469760000.0, 'train_loss': 0.05214171215856655, 'epoch': 3.0})

In [ ]:
import os

# Create the directory if it doesn't exist
os.makedirs("results", exist_ok=True)

# Save the model
model.save_pretrained("results/recipe_classifier")
# tokenizer.save_pretrained("results/recipe_classifier")

# Load the model
# from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
# model = DistilBertForSequenceClassification.from_pretrained("recipe_classifier")
# tokenizer = DistilBertTokenizer.from_pretrained("recipe_classifier")

('results/recipe_classifier/tokenizer_config.json',
 'results/recipe_classifier/special_tokens_map.json',
 'results/recipe_classifier/vocab.txt',
 'results/recipe_classifier/added_tokens.json',
 'results/recipe_classifier/tokenizer.json')

## Eval

In [ ]:
from torch.utils.data import Subset
from sklearn.metrics import classification_report
import numpy as np

# Take the first 100 items of the test dataset
subset_indices = list(range(50_000))
small_test_dataset = Subset(test_dataset, subset_indices)
small_test_labels = [test_labels[i] for i in subset_indices]

# Predict
preds_output = trainer.predict(small_test_dataset)
preds = np.argmax(preds_output.predictions, axis=1)

# Report
print(classification_report(
    small_test_labels,
    preds,
    target_names=['class1', 'class2'],
    zero_division=0
))


              precision    recall  f1-score   support

      class1       1.00      0.98      0.99     36080
      class2       0.95      0.99      0.97     13920

    accuracy                           0.98     50000
   macro avg       0.98      0.99      0.98     50000
weighted avg       0.98      0.98      0.98     50000

